In [129]:
import struct

import serial
import PIL
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import crc

In [102]:
digit_mask = PIL.Image.open('scans/digit.png')
seg_masks = [ np.asarray(PIL.Image.open('segmasks/seg{}.png'.format(i)), dtype=np.float)[:,:,0]/255.0 for i in range(1,9) ]
digit_h, digit_w = seg_masks[0].shape

In [193]:
pic = PIL.Image.open('circle.jpg')

In [194]:
dpi = 600
scale = 1000/5091
stride_x, stride_y = 1.0*dpi*scale, 38.8/25.4*dpi*scale

In [195]:
mask_w, mask_h = int(8*stride_x)+1, int(4*stride_y)+1
resized = np.asarray(pic.resize((mask_w, mask_h), PIL.Image.BILINEAR).convert('HSV'), dtype=np.float)[:,:,2]/255

In [196]:
def sub(resized, x, y):
    return resized[int(y*stride_y):int(y*stride_y)+digit_h, int(x*stride_x):int(x*stride_x)+digit_w]

In [197]:
def digit_values(resized, x, y):
    img = sub(resized, x, y)/255.0
    for mask in seg_masks:
        yield np.average(np.multiply(mask, img))

In [198]:
def make_vals(resized):
    subvals = np.zeros((8, 4, 8), dtype=np.float)
    for x in range(8):
        for y in range(4):
            subvals[x,y] = list(digit_values(resized, x, y))
    return subvals

In [204]:
def send_packet(ser, resized, addr=5):
    gamma = 2
    out = np.clip(np.power(make_vals(resized), 1/gamma)*255, 0, 255).astype(np.uint8)
    pkt = b''
    for x in range(8):
        for y in range(4):
            pkt += bytes(out[x,y,:]) + b'\x00\x00'
    pkt += b'\x03\x00\x00\x00'
    pkt += struct.pack('I', crc.crc(pkt))
    pkt = bytes([0x40 | addr]) + pkt
    ser.write(pkt)
    return out

In [205]:
ser = serial.Serial('/dev/serial/by-id/usb-FTDI_FT232R_USB_UART_A90JRHXH-if00-port0', 2000000)
send_packet(ser, resized)

array([[[3, 3, 3, 3, 3, 3, 3, 1],
        [3, 3, 3, 3, 3, 3, 3, 1],
        [3, 3, 3, 3, 3, 3, 3, 1],
        [3, 3, 3, 3, 3, 3, 3, 1]],

       [[3, 3, 3, 3, 3, 3, 3, 1],
        [3, 3, 3, 3, 3, 3, 3, 0],
        [2, 3, 0, 3, 3, 3, 3, 1],
        [3, 3, 3, 3, 3, 3, 3, 1]],

       [[3, 3, 3, 3, 3, 3, 3, 1],
        [1, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 1]],

       [[3, 3, 3, 3, 3, 3, 3, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 2, 3, 3, 3, 3, 1]],

       [[3, 3, 3, 3, 3, 3, 3, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 2, 3, 3, 3, 3, 1]],

       [[3, 3, 3, 3, 3, 3, 3, 1],
        [1, 0, 2, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 1]],

       [[3, 3, 3, 3, 3, 3, 3, 1],
        [3, 3, 3, 3, 0, 3, 1, 1],
        [2, 0, 3, 2, 1, 3, 3, 1],
        [3, 3, 3, 3, 3, 3, 3, 1]],

       [[3, 3, 3, 3, 3, 3, 3, 1],
